Grader should be put into same folder as `.py` files of assignments.  
To be used with grader, `.py` script of student should generate a `.csv` file in the same folder as the script itself.  
  
Csv should not contain index or header (as it is stated so in the assignment). First column should be file name (as it was said so during the class). Index will be cleared automatically.  
If the header is included, final grade will be 0, as it goes against the assignment (`csv file that has 100 rows`). (To avoid this, delete or comment the block with corresponding comment in cell 4).

In [ ]:
import glob
import pandas as pd
import re

Define $X$ that is used in `at least X syllabi`.  
(As a fraction).

In [ ]:
X = 0.2

Run all `.py` files in current directory.

In [ ]:
for py in glob.glob('./*.py'):
    exec(open(py).read())

Create a Pandas DataFrame of ratings:

In [ ]:
rowsList = []

for csv in glob.glob('./*.csv'):
    
    features = pd.read_csv(csv)
    
    fail = 1
    
#     delete the following block to stop script from giving 0 as a final grade if csv contains not exactly 100 rows
# ----------------------
    if len(features) != 100:
        fail = 0
# ----------------------       
        
#     clear column if it contains indexes
    for clmn in features.columns:
        if all(features[clmn] == [x for x in range(len(features))]):
            features.drop(clmn, axis=1, inplace=True)
            
#     drop first column that should contain file names  
    features.drop(columns=features.columns[0], inplace=True)
    
    row = {'student': re.findall('\.\\\\features-retrieved-by-([a-zA-Z]+)\.csv', csv)[0], # name of student
           'totalFeatures': len(features.columns) - 1} # number of extracted features total
    
#     if some value is repeated in a column with a frequency of >=10 we assume
#         this value was used by student as "nan" (or it is too hardcoded to be taken into account)
#     10% was chosen here as we will be interested in features that are applied to 90+% of syllabi
#         (and so, if the column contains only 5% of "nan"s, we can ignore this)
    for clmn in features.columns:
        if features[clmn].value_counts(normalize=True)[0] >= 0.1:
            features.replace(features[clmn].value_counts(normalize=True).index[0], 'nan', inplace=True)
    
#     calculate percentage of non-nan values for each feature
    pcts = []
    for clmn in features.columns:
        pcts.append(sum(features[clmn].str.lower() != 'nan') / len(features))
    
#     save them to write to df later
    for i, pct in enumerate(pcts):
        row['Feature' + ('0' + str(i) if i < 10 else str(i)[0] + str(i)[1])] = pct
    
    rowsList.append(row)

    
ratings = pd.DataFrame(rowsList)

# number of features that are extracted for at least X syllabi
ratings.loc[:,'featuresForAtLeastX'] =(ratings[ratings.columns[:-2]] >= X).sum(axis=1)

# grade
ratings.loc[:, 'grade'] = fail * ((ratings[ratings.columns[-1]] > 4) * (70 + 10 * (ratings[ratings.columns[-1]] - 5)) \
                            + 5 * (ratings[ratings.columns[:-3]] >= 0.9).sum(axis=1))
ratings.loc[:, 'grade'] = ratings.loc[:, 'grade'].apply(lambda x: min(x, 110))

# reorder dfcolumns for readability
ratings = ratings[ratings.columns.values.tolist()[-4:] + ratings.columns.values.tolist()[:-4]]

# sort df by number of features for at least X syllabi and for number of total features if first value match
ratings = ratings.sort_values(by=['featuresForAtLeastX', 'totalFeatures'], ascending=False)
ratings = ratings.reset_index()
ratings = ratings.drop('index', axis=1)

In [ ]:
ratings